In [1]:
import pandas as pd

df3 = pd.read_sql_query("SELECT * FROM gvs.vw_ml_land_only_glm_ready_v3;", engine)

NameError: name 'engine' is not defined

In [2]:
from dotenv import load_dotenv
from pathlib import Path
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

load_dotenv(Path(r"C:\OneDrive\gvs-kyiv\env\.env"))

host = os.getenv("PGHOST", "localhost")
port = int(os.getenv("PGPORT", "5432"))
user = os.getenv("PGUSER")
pwd  = os.getenv("PGPASSWORD")
db   = os.getenv("PGDATABASE")

url = f"postgresql+psycopg2://{user}:{quote_plus(pwd)}@{host}:{port}/{db}"
engine = create_engine(url, pool_pre_ping=True)

In [3]:
import pandas as pd

df3 = pd.read_sql_query("SELECT * FROM gvs.vw_ml_land_only_glm_ready_v3;", engine)

In [5]:
df3.shape, df3.columns.tolist()

((52, 16),
 ['spatial_unit_id',
  'y_ln_uah_m2',
  'x_ln_area',
  'lon',
  'lat',
  'x_ln_dist_metro',
  'x_ln_dist_school',
  'x_ln_dist_kindergarten',
  'x_ln_dist_park',
  'x_ln_dist_water',
  'x_ln_dist_cemetery',
  'x_ln_dist_highway',
  'x_ln_dist_powerline',
  'gp_band_1',
  'gp_band_2',
  'gp_band_3'])

In [6]:
pd.read_sql_query("""
select column_name
from information_schema.columns
where table_schema='gvs' and table_name='vw_ml_land_only_glm_ready_v2'
order by ordinal_position;
""", engine)

,column_name
0,spatial_unit_id
1,y_ln_uah_m2
2,lon
3,lat
4,x_ln_area
5,x_ln_dist_metro
6,x_ln_dist_school
7,x_ln_dist_kindergarten
8,x_ln_dist_park
9,x_ln_dist_water


In [7]:
base_cols = [
    "x_ln_area",
    "lon", "lat",
    "x_ln_dist_metro",
    "x_ln_dist_school",
    "x_ln_dist_kindergarten",
    "x_ln_dist_park",
    "x_ln_dist_water",
    "x_ln_dist_cemetery",
    "x_ln_dist_highway",
    "x_ln_dist_powerline",
    "gp_band_1", "gp_band_2", "gp_band_3",
]


In [8]:
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

y = df3["y_ln_uah_m2"].astype(float)
X_base = df3[base_cols].astype(float)
X_base = sm.add_constant(X_base)

m_base = sm.OLS(y, X_base).fit(cov_type="HC3")
print(m_base.summary())


                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     11.93
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           9.59e-10
Time:                        23:40:04   Log-Likelihood:                -25.809
No. Observations:                  52   AIC:                             81.62
Df Residuals:                      37   BIC:                             110.9
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -37

In [9]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
y_true, y_pred = [], []

for tr, te in loo.split(X_base):
    m = sm.OLS(y.iloc[tr], X_base.iloc[tr]).fit()
    y_hat = m.predict(X_base.iloc[te])
    y_true.append(y.iloc[te].values[0])
    y_pred.append(y_hat.values[0])

y_true = np.array(y_true)
y_pred = np.array(y_pred)

r2_log_base = r2_score(y_true, y_pred)
mae_base = mean_absolute_error(np.exp(y_true), np.exp(y_pred))
mape_base = np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

r2_log_base, mae_base, mape_base


(0.41766724704854974, 3341.991557910467, np.float64(43.7652935952949))

In [10]:
osm_cols = [c for c in df3.columns if c.startswith("x_ln_osm_")]
cnt_cols = [c for c in df3.columns if c.startswith("x_ln_cnt500_")]

full_cols = base_cols + osm_cols + cnt_cols


In [11]:
X_full = df3[full_cols].astype(float)
X_full = sm.add_constant(X_full)

m_full = sm.OLS(y, X_full).fit(cov_type="HC3")
print(m_full.summary())


                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     11.93
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           9.59e-10
Time:                        23:41:38   Log-Likelihood:                -25.809
No. Observations:                  52   AIC:                             81.62
Df Residuals:                      37   BIC:                             110.9
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -37

In [12]:
df3 = pd.read_sql_query("SELECT * FROM gvs.vw_ml_land_only_glm_ready_v3;", engine)
print(df3.shape)
print(df3.columns.tolist())


(52, 16)
['spatial_unit_id', 'y_ln_uah_m2', 'x_ln_area', 'lon', 'lat', 'x_ln_dist_metro', 'x_ln_dist_school', 'x_ln_dist_kindergarten', 'x_ln_dist_park', 'x_ln_dist_water', 'x_ln_dist_cemetery', 'x_ln_dist_highway', 'x_ln_dist_powerline', 'gp_band_1', 'gp_band_2', 'gp_band_3']


In [13]:
print(df3.shape)
print(df3.columns.tolist())

(52, 16)
['spatial_unit_id', 'y_ln_uah_m2', 'x_ln_area', 'lon', 'lat', 'x_ln_dist_metro', 'x_ln_dist_school', 'x_ln_dist_kindergarten', 'x_ln_dist_park', 'x_ln_dist_water', 'x_ln_dist_cemetery', 'x_ln_dist_highway', 'x_ln_dist_powerline', 'gp_band_1', 'gp_band_2', 'gp_band_3']


In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import LeaveOneOut

df = pd.read_sql_query("SELECT * FROM gvs.vw_ml_land_only_glm_ready_v3;", engine)

y = df["y_ln_uah_m2"].astype(float)
X = df.drop(columns=["spatial_unit_id","y_ln_uah_m2"]).astype(float)
X = sm.add_constant(X, has_constant="add")

m = sm.OLS(y, X).fit(cov_type="HC3")
print(m.summary())

# in-sample metrics
yhat = m.predict(X)
r2_log = r2_score(y, yhat)

y_true = np.exp(y)
y_pred = np.exp(yhat)
mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print("R2_log:", r2_log, "MAE:", mae, "MAPE%:", mape)

# LOOCV
loo = LeaveOneOut()
y_true_lo, y_pred_lo = [], []

for tr, te in loo.split(X):
    m_i = sm.OLS(y.iloc[tr], X.iloc[tr]).fit()
    y_pred_i = m_i.predict(X.iloc[te]).values[0]
    y_true_lo.append(y.iloc[te].values[0])
    y_pred_lo.append(y_pred_i)

y_true_lo = np.array(y_true_lo)
y_pred_lo = np.array(y_pred_lo)

print("LOOCV R2_log:", r2_score(y_true_lo, y_pred_lo))
print("LOOCV MAE:", mean_absolute_error(np.exp(y_true_lo), np.exp(y_pred_lo)))
print("LOOCV MAPE%:", np.mean(np.abs((np.exp(y_true_lo)-np.exp(y_pred_lo))/np.exp(y_true_lo))) * 100)


                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.491
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     2.924
Date:                Mon, 19 Jan 2026   Prob (F-statistic):            0.00454
Time:                        00:55:12   Log-Likelihood:                -86.619
No. Observations:                  52   AIC:                             203.2
Df Residuals:                      37   BIC:                             232.5
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   -654

In [15]:
df = pd.read_sql("""
    SELECT *
    FROM gvs.spatial_unit_v3_empty
    JOIN gvs.vw_ml_land_only_glm_ready_v3 USING (spatial_unit_id)
""", engine)


In [16]:
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, r2_score

# target
y_log = df["y_ln_uah_m2"].astype(float)

# features (візьми всі x_* + lon/lat + gp_band_*)
feature_cols = [c for c in df.columns if c.startswith("x_")] + ["lon","lat","gp_band_1","gp_band_2","gp_band_3"]
X = df[feature_cols].astype(float)

X = sm.add_constant(X, has_constant="add")
model = sm.OLS(y_log, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     7.399
Date:                Mon, 19 Jan 2026   Prob (F-statistic):           5.22e-07
Time:                        01:23:22   Log-Likelihood:                -25.809
No. Observations:                  52   AIC:                             81.62
Df Residuals:                      37   BIC:                             110.9
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -37

In [17]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
y_true, y_pred = [], []

for tr, te in loo.split(X_base):
    m = sm.OLS(y.iloc[tr], X_base.iloc[tr]).fit()
    y_hat = m.predict(X_base.iloc[te])
    y_true.append(y.iloc[te].values[0])
    y_pred.append(y_hat.values[0])

y_true = np.array(y_true)
y_pred = np.array(y_pred)

r2_log_base = r2_score(y_true, y_pred)
mae_base = mean_absolute_error(np.exp(y_true), np.exp(y_pred))
mape_base = np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

r2_log_base, mae_base, mape_base

(-0.2066874457129113, 136604.28569997274, np.float64(227.87160963627434))

In [18]:
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

y = df["y_ln_uah_m2"].astype(float)
X_base = df[base_cols].astype(float)
X_base = sm.add_constant(X_base)

m_base = sm.OLS(y, X_base).fit(cov_type="HC3")
print(m_base.summary())

                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     11.93
Date:                Mon, 19 Jan 2026   Prob (F-statistic):           9.59e-10
Time:                        01:25:21   Log-Likelihood:                -25.809
No. Observations:                  52   AIC:                             81.62
Df Residuals:                      37   BIC:                             110.9
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -37

In [19]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
y_true, y_pred = [], []

for tr, te in loo.split(X_base):
    m = sm.OLS(y.iloc[tr], X_base.iloc[tr]).fit()
    y_hat = m.predict(X_base.iloc[te])
    y_true.append(y.iloc[te].values[0])
    y_pred.append(y_hat.values[0])

y_true = np.array(y_true)
y_pred = np.array(y_pred)

r2_log_base = r2_score(y_true, y_pred)
mae_base = mean_absolute_error(np.exp(y_true), np.exp(y_pred))
mape_base = np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

r2_log_base, mae_base, mape_base

(0.41766724704854974, 3341.991557910467, np.float64(43.7652935952949))

In [22]:
osm_cols = [c for c in df.columns if c.startswith("x_ln_osm_")]
cnt_cols = [c for c in df.columns if c.startswith("x_ln_cnt500_")]

full_cols = base_cols + osm_cols + cnt_cols

In [23]:
X_full = df[full_cols].astype(float)
X_full = sm.add_constant(X_full)

m_full = sm.OLS(y, X_full).fit(cov_type="HC3")
print(m_full.summary())


                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     11.93
Date:                Mon, 19 Jan 2026   Prob (F-statistic):           9.59e-10
Time:                        01:27:00   Log-Likelihood:                -25.809
No. Observations:                  52   AIC:                             81.62
Df Residuals:                      37   BIC:                             110.9
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -37

In [25]:
# приклад "OSM замість dist"
base_cols_no_overlap = [c for c in base_cols if c not in [
    "x_ln_dist_school","x_ln_dist_kindergarten","x_ln_dist_metro","x_ln_dist_park"
]]

X_osm = df[base_cols_no_overlap + osm_cols + cnt_cols].astype(float)

# прибрати константні колонки (часта причина сингулярності)
const_like = [c for c in X_osm.columns if X_osm[c].nunique(dropna=True) <= 1]
X_osm = X_osm.drop(columns=const_like)

X_osm = sm.add_constant(X_osm, has_constant="add")
m_osm = sm.OLS(y, X_osm).fit(cov_type="HC3")
print("dropped const:", const_like)
print(m_osm.summary())

dropped const: []
                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.541
Model:                            OLS   Adj. R-squared:                  0.429
Method:                 Least Squares   F-statistic:                     5.591
Date:                Mon, 19 Jan 2026   Prob (F-statistic):           3.25e-05
Time:                        01:27:42   Log-Likelihood:                -40.281
No. Observations:                  52   AIC:                             102.6
Df Residuals:                      41   BIC:                             124.0
Df Model:                          10                                         
Covariance Type:                  HC3                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const           

In [27]:
full_cols = [c for c in df.columns if c not in ["spatial_unit_id","y_ln_uah_m2"]]
X_full = df[full_cols].astype(float)

const_like = [c for c in X_full.columns if X_full[c].nunique(dropna=True) <= 1]
X_full = X_full.drop(columns=const_like)

X_full = sm.add_constant(X_full, has_constant="add")
m_full = sm.OLS(y, X_full).fit(cov_type="HC3")
print("dropped const:", const_like)
print(m_full.summary())

ValueError: could not convert string to float: '8000000000:91:023:0009'

In [1]:
df = pd.read_sql("""
    SELECT *
    FROM gvs.vw_market_agg_v3_residual_v1
    JOIN gvs.vw_market_agg_v3 USING (spatial_unit_id)
""", engine)

NameError: name 'pd' is not defined

In [2]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM gvs.vw_market_agg_v3_residual_v1;", engine)

NameError: name 'engine' is not defined

In [3]:
from dotenv import load_dotenv
from pathlib import Path
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

load_dotenv(Path(r"C:\OneDrive\gvs-kyiv\env\.env"))

host = os.getenv("PGHOST", "localhost")
port = int(os.getenv("PGPORT", "5432"))
user = os.getenv("PGUSER")
pwd  = os.getenv("PGPASSWORD")
db   = os.getenv("PGDATABASE")

url = f"postgresql+psycopg2://{user}:{quote_plus(pwd)}@{host}:{port}/{db}"
engine = create_engine(url, pool_pre_ping=True)

In [4]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM gvs.vw_market_agg_v3_residual_v1;", engine)

In [5]:
df = pd.read_sql("""
    SELECT *
    FROM gvs.vw_market_agg_v3_residual_v1
    JOIN gvs.vw_market_agg_v3 USING (spatial_unit_id)
""", engine)

ProgrammingError: (psycopg2.errors.UndefinedTable) ПОМИЛКА:  відношення "gvs.vw_market_agg_v3" не існує
LINE 4:     JOIN gvs.vw_market_agg_v3 USING (spatial_unit_id)
                 ^

[SQL: 
    SELECT *
    FROM gvs.vw_market_agg_v3_residual_v1
    JOIN gvs.vw_market_agg_v3 USING (spatial_unit_id)
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [6]:
df = pd.read_sql_query("SELECT * FROM gvs.vw_market_agg_v3_residual_v1;", engine)
df.shape, df.columns.tolist()[:20]


((144, 8),
 ['spatial_unit_id',
  'cad_number',
  'real_area_m2',
  'total_value_uah_med',
  'total_price_uah_m2_med',
  'estimated_improvement_value_uah',
  'land_value_residual_uah',
  'land_price_residual_uah_m2'])

In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import LeaveOneOut, train_test_split

df = pd.read_sql_query("SELECT * FROM gvs.vw_ml_purpose5_residual_glm_ready_v1;", engine)
df.shape, df.columns.tolist()


((141, 32),
 ['spatial_unit_id',
  'cad_number',
  'real_area_m2',
  'y_ln_land_uah_m2',
  'lon',
  'lat',
  'x_ln_area',
  'x_ln_dist_metro',
  'x_ln_dist_school',
  'x_ln_dist_kindergarten',
  'x_ln_dist_park',
  'x_ln_dist_water',
  'x_ln_dist_cemetery',
  'x_ln_dist_highway',
  'x_ln_dist_powerline',
  'x_ln_dist_pt',
  'x_ln_dist_supermarket',
  'x_ln_dist_hospital',
  'x_ln_dist_railway',
  'x_ln_dist_mall',
  'x_ln_dist_wastewater',
  'x_ln_dist_incineration',
  'x_ln_dist_beach',
  'x_ln_dist_chp',
  'gp_band_1',
  'gp_band_2',
  'gp_band_3',
  'land_price_residual_uah_m2',
  'land_value_residual_uah',
  'total_value_uah_med',
  'total_price_uah_m2_med',
  'estimated_improvement_value_uah'])

In [8]:
target = "y_ln_land_uah_m2"

drop_cols = {
    "spatial_unit_id","cad_number","real_area_m2",
    target,
    # audit:
    "land_price_residual_uah_m2","land_value_residual_uah",
    "total_value_uah_med","total_price_uah_m2_med","estimated_improvement_value_uah"
}

feature_cols = [c for c in df.columns if c not in drop_cols]

X = df[feature_cols].astype(float)
X = sm.add_constant(X, has_constant="add")
y = df[target].astype(float)

model = sm.OLS(y, X).fit(cov_type="HC3")
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:       y_ln_land_uah_m2   R-squared:                       0.526
Model:                            OLS   Adj. R-squared:                  0.433
Method:                 Least Squares   F-statistic:                     8.776
Date:                Mon, 19 Jan 2026   Prob (F-statistic):           2.47e-16
Time:                        08:14:45   Log-Likelihood:                -144.03
No. Observations:                 141   AIC:                             336.1
Df Residuals:                     117   BIC:                             406.8
Df Model:                          23                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -98

In [9]:
y_pred_log = model.predict(X)

r2_log = r2_score(y, y_pred_log)

y_true = np.exp(y)
y_pred = np.exp(y_pred_log)

mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print({"R2_log": r2_log, "MAE_uah_m2": mae, "MAPE_%": mape})


{'R2_log': 0.5263916610735953, 'MAE_uah_m2': 8642.62716462106, 'MAPE_%': np.float64(58.43913674211674)}


In [10]:
loo = LeaveOneOut()
y_true_log, y_pred_log = [], []

for tr, te in loo.split(X):
    m = sm.OLS(y.iloc[tr], X.iloc[tr]).fit()
    yhat = m.predict(X.iloc[te]).values[0]
    y_true_log.append(y.iloc[te].values[0])
    y_pred_log.append(yhat)

y_true_log = np.array(y_true_log)
y_pred_log = np.array(y_pred_log)

r2_log_loo = r2_score(y_true_log, y_pred_log)

y_true_loo = np.exp(y_true_log)
y_pred_loo = np.exp(y_pred_log)

mae_loo = mean_absolute_error(y_true_loo, y_pred_loo)
mape_loo = np.mean(np.abs((y_true_loo - y_pred_loo) / y_true_loo)) * 100

print({"R2_log_LOO": r2_log_loo, "MAE_uah_m2_LOO": mae_loo, "MAPE_%_LOO": mape_loo})


{'R2_log_LOO': 0.29916674820159406, 'MAE_uah_m2_LOO': 9835.305099851743, 'MAPE_%_LOO': np.float64(73.55432938940494)}


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1) тягнемо навчальний датасет (він у тебе вже валідний: bad_y/bad_area/bad_xy = 0)
df = pd.read_sql_query("SELECT * FROM gvs.vw_ml_purpose5_residual_glm_ready_v1;", engine)

target = "y_ln_land_uah_m2"

# 2) фічі = все, що НЕ id/аудит/ціль
drop_cols = {
    "spatial_unit_id","cad_number","real_area_m2",
    target,
    # audit fields (якщо вони присутні в df)
    "land_price_residual_uah_m2","land_value_residual_uah",
    "total_value_uah_med","total_price_uah_m2_med","estimated_improvement_value_uah"
}
feature_cols = [c for c in df.columns if c not in drop_cols]

X = df[feature_cols].astype(float)
X = sm.add_constant(X, has_constant="add")
y = df[target].astype(float)

m = sm.OLS(y, X).fit(cov_type="HC3")

df_pred = pd.DataFrame({
    "spatial_unit_id": df["spatial_unit_id"].astype("int64"),
    "y_ln_land_uah_m2_true": y.values,
    "y_ln_land_uah_m2_pred": m.predict(X).values,
})
df_pred["land_price_uah_m2_true"] = np.exp(df_pred["y_ln_land_uah_m2_true"])
df_pred["land_price_uah_m2_pred"] = np.exp(df_pred["y_ln_land_uah_m2_pred"])

# 3) пишемо в БД (таблиця прогнозів)
# важливо: якщо таблиця вже є — перезаписуємо її
df_pred.to_sql(
    "ml_purpose5_residual_predictions_v1",
    engine,
    schema="gvs",
    if_exists="replace",
    index=False,
)


NameError: name 'engine' is not defined

In [ ]:
df = pd.read_sql_query("SELECT * FROM gvs.vw_market_agg_v3_residual_v1;", engine)



In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1) тягнемо навчальний датасет (він у тебе вже валідний: bad_y/bad_area/bad_xy = 0)
df = pd.read_sql_query("SELECT * FROM gvs.vw_ml_purpose5_residual_glm_ready_v1;", engine)

target = "y_ln_land_uah_m2"

# 2) фічі = все, що НЕ id/аудит/ціль
drop_cols = {
    "spatial_unit_id","cad_number","real_area_m2",
    target,
    # audit fields (якщо вони присутні в df)
    "land_price_residual_uah_m2","land_value_residual_uah",
    "total_value_uah_med","total_price_uah_m2_med","estimated_improvement_value_uah"
}
feature_cols = [c for c in df.columns if c not in drop_cols]

X = df[feature_cols].astype(float)
X = sm.add_constant(X, has_constant="add")
y = df[target].astype(float)

m = sm.OLS(y, X).fit(cov_type="HC3")

df_pred = pd.DataFrame({
    "spatial_unit_id": df["spatial_unit_id"].astype("int64"),
    "y_ln_land_uah_m2_true": y.values,
    "y_ln_land_uah_m2_pred": m.predict(X).values,
})
df_pred["land_price_uah_m2_true"] = np.exp(df_pred["y_ln_land_uah_m2_true"])
df_pred["land_price_uah_m2_pred"] = np.exp(df_pred["y_ln_land_uah_m2_pred"])

# 3) пишемо в БД (таблиця прогнозів)
# важливо: якщо таблиця вже є — перезаписуємо її
df_pred.to_sql(
    "ml_purpose5_residual_predictions_v1",
    engine,
    schema="gvs",
    if_exists="replace",
    index=False,
)


NameError: name 'engine' is not defined

In [3]:
from dotenv import load_dotenv
from pathlib import Path
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

load_dotenv(Path(r"C:\OneDrive\gvs-kyiv\env\.env"))

host = os.getenv("PGHOST", "localhost")
port = int(os.getenv("PGPORT", "5432"))
user = os.getenv("PGUSER")
pwd  = os.getenv("PGPASSWORD")
db   = os.getenv("PGDATABASE")

url = f"postgresql+psycopg2://{user}:{quote_plus(pwd)}@{host}:{port}/{db}"
engine = create_engine(url, pool_pre_ping=True)

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1) тягнемо навчальний датасет (він у тебе вже валідний: bad_y/bad_area/bad_xy = 0)
df = pd.read_sql_query("SELECT * FROM gvs.vw_ml_purpose5_residual_glm_ready_v1;", engine)

target = "y_ln_land_uah_m2"

# 2) фічі = все, що НЕ id/аудит/ціль
drop_cols = {
    "spatial_unit_id","cad_number","real_area_m2",
    target,
    # audit fields (якщо вони присутні в df)
    "land_price_residual_uah_m2","land_value_residual_uah",
    "total_value_uah_med","total_price_uah_m2_med","estimated_improvement_value_uah"
}
feature_cols = [c for c in df.columns if c not in drop_cols]

X = df[feature_cols].astype(float)
X = sm.add_constant(X, has_constant="add")
y = df[target].astype(float)

m = sm.OLS(y, X).fit(cov_type="HC3")

df_pred = pd.DataFrame({
    "spatial_unit_id": df["spatial_unit_id"].astype("int64"),
    "y_ln_land_uah_m2_true": y.values,
    "y_ln_land_uah_m2_pred": m.predict(X).values,
})
df_pred["land_price_uah_m2_true"] = np.exp(df_pred["y_ln_land_uah_m2_true"])
df_pred["land_price_uah_m2_pred"] = np.exp(df_pred["y_ln_land_uah_m2_pred"])

# 3) пишемо в БД (таблиця прогнозів)
# важливо: якщо таблиця вже є — перезаписуємо її
df_pred.to_sql(
    "ml_purpose5_residual_predictions_v1",
    engine,
    schema="gvs",
    if_exists="replace",
    index=False,
)

141